In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

Cargo los dataframes originales

In [2]:
df_original = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

Me quedo con las columnas de interes

In [3]:
df_reducido = df_original.loc[:,['person','event','sku','model','city','region','country']]
df_reducido.head()

,person,event,sku,model,city,region,country
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,NaN,NaN,NaN
1,ad93850f,viewed product,304.0,iPhone 5s,NaN,NaN,NaN
2,0297fc1e,viewed product,6888.0,iPhone 6S,NaN,NaN,NaN
3,2d681dd8,viewed product,11890.0,iPhone 7,NaN,NaN,NaN
4,cccea85e,viewed product,7517.0,LG G4 H818P,NaN,NaN,NaN


Construyo un dataframe que tiene las personas, con las regiones a las que pertenecen, y solo las que realizaron viewed product

In [4]:
df_region_person = df_original.loc[df_original['country'].notnull(),['person','region','country']]

In [5]:
df_region_person['counter'] = 1
df_prueba = df_region_person.groupby(['person','region'])['counter'].sum().reset_index()

In [6]:
df_region_person= df_prueba.sort_values(by='counter',ascending = False).drop_duplicates(subset='person',keep='first')

In [7]:
df_region_person = df_region_person.rename(columns={'region':'most_visited_region'})

In [8]:
df_region_person = df_region_person.drop(['counter'],axis=1)

In [9]:
df_reducido = df_reducido.merge(df_region_person, left_on='person', right_on='person', how='left')
df_reducido.head()

,person,event,sku,model,city,region,country,most_visited_region
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,NaN,NaN,NaN,Rio de Janeiro
1,ad93850f,viewed product,304.0,iPhone 5s,NaN,NaN,NaN,Sao Paulo
2,0297fc1e,viewed product,6888.0,iPhone 6S,NaN,NaN,NaN,Rio de Janeiro
3,2d681dd8,viewed product,11890.0,iPhone 7,NaN,NaN,NaN,Sao Paulo
4,cccea85e,viewed product,7517.0,LG G4 H818P,NaN,NaN,NaN,Sao Paulo


In [10]:
df_reducido = df_reducido.loc[df_reducido['sku'].notnull(),['person','event','sku','model','most_visited_region']]
df_reducido.head()

,person,event,sku,model,most_visited_region
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,Rio de Janeiro
1,ad93850f,viewed product,304.0,iPhone 5s,Sao Paulo
2,0297fc1e,viewed product,6888.0,iPhone 6S,Rio de Janeiro
3,2d681dd8,viewed product,11890.0,iPhone 7,Sao Paulo
4,cccea85e,viewed product,7517.0,LG G4 H818P,Sao Paulo


In [11]:
df_reducido.person.drop_duplicates().shape

(38250,)

Ahora tengo un dataframe que tiene por persona, la region a la que pertenece y los modelos con los que interactuo. Ahora, solo quiero los que miro:

In [12]:
df_only_vp = df_reducido.loc[df_reducido['event'] == 'viewed product', :]

Ahora quiero, por persona, el producto que mas visito:

In [13]:
df_most_visited_per_person = df_only_vp.groupby(['person','model']).size().reset_index()\
                                                                            .rename(columns={0:'Cantidad_visitas'})

In [14]:
df_most_visited_per_person = df_most_visited_per_person.sort_values(by='Cantidad_visitas',ascending=False)\
                                                        .drop_duplicates(subset='person',keep='first')

In [15]:
df_most_visited_per_person = df_most_visited_per_person.merge(df_region_person, on='person',how='left')

In [16]:
df_most_visited_per_person.head()

,person,model,Cantidad_visitas,most_visited_region
0,b1f4dbf6,iPhone 5s,832,Sao Paulo
1,171e75cb,iPhone 6S,679,Espirito Santo
2,6f19cfd9,iPhone 6S Plus,622,Bahia
3,6abd2bf1,iPhone 6,603,Sao Paulo
4,9bf968c5,iPhone 6,558,Bahia


Ahora, un dataframe que tiene el top 5 de cada region de los productos mas visitados.

In [17]:
df_most_visited_per_region = df_original.loc[df_original['event'] == 'viewed product', ['person','region', 'model', 'sku']]
df_most_visited_per_region = df_most_visited_per_region.drop(columns='region')
df_most_visited_per_region = df_most_visited_per_region.merge(df_region_person, on='person', how='left')
df_most_visited_per_region = df_most_visited_per_region.rename(columns={'most_visited_region':'region'})
df_most_visited_per_region.head()

,person,model,sku,region
0,4886f805,Samsung Galaxy J7 Prime,9288.0,Rio de Janeiro
1,ad93850f,iPhone 5s,304.0,Sao Paulo
2,0297fc1e,iPhone 6S,6888.0,Rio de Janeiro
3,2d681dd8,iPhone 7,11890.0,Sao Paulo
4,cccea85e,LG G4 H818P,7517.0,Sao Paulo


Aca calculo la cantidad de visitas por producto y por regions

In [18]:
df_most_visited_per_region = df_most_visited_per_region.groupby(['region','model']).size().reset_index().rename(columns={0:'Visitas'})

In [19]:
df_most_visited_per_region.sort_values(by='Visitas',ascending=False).head()

,region,model,Visitas
4358,Sao Paulo,iPhone 6,30010
4360,Sao Paulo,iPhone 6S,27908
4357,Sao Paulo,iPhone 5s,26446
4362,Sao Paulo,iPhone 7,18638
4955,Unknown,iPhone 6,15574


Ahora, solo conservamos el top 5 de cada una de las regiones

In [20]:
df_most_visited_per_region = df_most_visited_per_region.sort_values(by='Visitas',ascending=False)\
                                                       .groupby('region').head(5)

Construyo un diccionario que tiene como clave la region, y como valor una lista con los 5 productos mas visitados por los usuarios de esa region (algunas listas tienen menos de 5 items por la falta de datos de algunas regiones)

In [21]:
df_top_5_por_region = df_most_visited_per_region.groupby('region')
top_5_region = {}
regiones_del_df = df_most_visited_per_region['region'].drop_duplicates().dropna()
for region in regiones_del_df:
    top_5_region[region] = list(df_top_5_por_region.get_group(region)['model'])

In [22]:
top_5_region['Sao Paulo']

['iPhone 6', 'iPhone 6S', 'iPhone 5s', 'iPhone 7', 'Samsung Galaxy S7 Edge']

Ahora, utilizando ese diccionario y el dataframe del producto mas visitado por cada persona, construyo una columna que decide si la persona visitó mas veces un producto que entra en el top 5 de su region.

In [23]:
df_most_visited_per_person = df_most_visited_per_person.dropna()
df_most_visited_per_person['person'].describe()


count        36931
unique       36931
top       8808be0f
freq             1
Name: person, dtype: object

In [24]:
df_most_visited_per_person['busco_top_5_visitas'] = [s.model in top_5_region[s.most_visited_region] for s in df_most_visited_per_person.itertuples()]

In [25]:
df_most_visited_per_person.head(10)

,person,model,Cantidad_visitas,most_visited_region,busco_top_5_visitas
0,b1f4dbf6,iPhone 5s,832,Sao Paulo,True
1,171e75cb,iPhone 6S,679,Espirito Santo,True
2,6f19cfd9,iPhone 6S Plus,622,Bahia,True
3,6abd2bf1,iPhone 6,603,Sao Paulo,True
4,9bf968c5,iPhone 6,558,Bahia,True
5,656c18ef,iPhone 6S,533,Unknown,True
6,eb6c498c,iPhone 6S,500,Minas Gerais,True
7,0d5f66ed,iPhone 5s,495,Alagoas,True
8,0dbcf50c,iPhone SE,486,Unknown,False
9,97b0c0d1,iPhone SE,477,Sao Paulo,False


In [26]:
df_final = df_most_visited_per_person